<a href="https://colab.research.google.com/github/vanessabrolim/comfort_analysis/blob/master/consenso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from scipy import stats
from geopy.distance import vincenty
from google.colab import drive
drive.mount('/content/drive',  force_remount=True)

Mounted at /content/drive


In [0]:
df_questionario = pd.read_csv('drive/My Drive/Udesc/questionarios.csv', 
                              low_memory = False)

In [0]:
df_questionario.tail()

,Data,Entrada,Saída,z,y,x,Geral
212,25/09/2017,12:30:00,12:45:00,2,4,2,4
213,25/09/2017,12:30:00,12:45:00,3,3,4,4
214,25/09/2017,12:30:00,12:45:00,3,4,3,4
215,25/09/2017,12:30:00,12:45:00,4,3,3,4
216,25/09/2017,12:30:00,12:45:00,3,4,3,4


In [0]:
df_questionario['Entrada'] = pd.to_datetime(df_questionario['Data']+ ' ' + df_questionario['Entrada'])


In [0]:
df_questionario['Saída'] = pd.to_datetime(df_questionario['Data']+ ' ' + df_questionario['Saída'])

In [0]:
df_questionario.dtypes

Data               object
Entrada    datetime64[ns]
Saída      datetime64[ns]
z                   int64
y                   int64
x                   int64
Geral               int64
dtype: object

In [0]:
df_questionario.head()

,Data,Entrada,Saída,z,y,x,Geral
0,13/09/2017,2017-09-13 11:21:00,2017-09-13 11:36:00,5,5,2,4
1,13/09/2017,2017-09-13 11:20:00,2017-09-13 11:35:00,4,4,1,4
2,13/09/2017,2017-09-13 11:50:00,2017-09-13 12:05:00,5,3,3,5
3,13/09/2017,2017-09-13 12:15:00,2017-09-13 12:30:00,3,3,2,4
4,13/09/2017,2017-09-13 12:10:00,2017-09-13 12:25:00,4,4,4,4


In [0]:
df_questionario = df_questionario.sort_values(by=['Entrada']).reset_index(drop=True)

In [0]:
df_questionario.head()

,Data,Entrada,Saída,z,y,x,Geral
0,13/09/2017,2017-09-13 08:47:00,2017-09-13 09:02:00,5,4,5,4
1,13/09/2017,2017-09-13 08:47:00,2017-09-13 09:02:00,3,4,4,4
2,13/09/2017,2017-09-13 08:47:00,2017-09-13 09:02:00,3,3,4,3
3,13/09/2017,2017-09-13 09:01:00,2017-09-13 09:16:00,4,4,2,4
4,13/09/2017,2017-09-13 11:15:00,2017-09-13 11:30:00,2,3,1,3


In [0]:
inicio = df_questionario['Entrada'][0]
fim = df_questionario['Saída'][0]
notas_x = []
notas_y = []
notas_z = []
notas_geral = []
df_q = pd.DataFrame(columns=('Início', 'Fim', 'x', 'y', 'z', 'Geral'))
df_q = df_q.astype(dtype={'Início': 'datetime64[ns]',
                          'Fim': 'datetime64[ns]', 
                          'x': 'int64', 
                          'y': 'int64', 
                          'z': 'int64', 
                          'Geral': 'int64'})

In [0]:
for index, row in df_questionario.iterrows():
  if (row['Entrada'] >= inicio and row['Entrada'] < fim):
    notas_x.append(row['x'])
    notas_y.append(row['y'])
    notas_z.append(row['z'])
    notas_geral.append(row['Geral'])
    if (row['Saída'] > fim):
      df_q = df_q.append({'Início': inicio, 
                          'Fim' : fim, 
                          'x' : round(np.mean(notas_x)), 
                          'y' : round(np.mean(notas_y)), 
                          'z' : round(np.mean(notas_z)), 
                          'Geral' : round(np.mean(notas_geral))}, 
                         ignore_index=True)
      notas_x = []
      notas_y = []
      notas_z = []
      notas_geral = []
      inicio = fim
      fim = row['Saída']
  else:
    notas_x.append(row['x'])
    notas_y.append(row['y'])
    notas_z.append(row['z'])
    notas_geral.append(row['Geral'])
    df_q = df_q.append({'Início': inicio, 
                        'Fim' : fim, 
                        'x' : round(np.mean(notas_x)), 
                        'y' : round(np.mean(notas_y)), 
                        'z' : round(np.mean(notas_z)), 
                        'Geral' : round(np.mean(notas_geral))}, 
                       ignore_index=True)
    notas_x = []
    notas_y = []
    notas_z = []
    notas_geral = []
    inicio = row['Entrada'] 
    fim = row['Saída']
    


In [0]:
df_q = df_q.sort_values(by=['Início']).reset_index(drop=True)


In [0]:
inicio = df_q['Início'][0]
fim = df_q['Fim'][0] 
notas_x = []
notas_y = []
notas_z = []
notas_geral = []
df_quest = pd.DataFrame(columns=('Início', 'Fim', 'x', 'y', 'z', 'Geral'))
df_quest = df_quest.astype(dtype={'Início': 'datetime64[ns]',
                                  'Fim': 'datetime64[ns]',
                                  'x': 'int64', 
                                  'y': 'int64', 
                                  'z': 'int64',
                                  'Geral': 'int64'})

In [0]:
for index, row in df_q.iterrows():
  if (row['Início'] >= inicio and row['Início']< fim):
    notas_x.append(row['x'])
    notas_y.append(row['y'])
    notas_z.append(row['z'])
    notas_geral.append(row['Geral'])
    if (row['Fim'] > fim):
      df_quest = df_quest.append({'Início': inicio, 'Fim' : fim, 
                                  'x' : round(np.mean(notas_x)), 
                                  'y' : round(np.mean(notas_y)), 
                                  'z' : round(np.mean(notas_z)), 
                                  'Geral' : round(np.mean(notas_geral))}, 
                                 ignore_index=True)
      notas_x = []
      notas_y = []
      notas_z = []
      notas_geral = []
      inicio = fim
      fim = row['Fim'] 
  else:
    notas_x.append(row['x'])
    notas_y.append(row['y'])
    notas_z.append(row['z'])
    notas_geral.append(row['Geral'])
    df_quest = df_quest.append({'Início': inicio, 
                                'Fim' : fim, 
                                'x' : round(np.mean(notas_x)), 
                                'y' : round(np.mean(notas_y)), 
                                'z' : round(np.mean(notas_z)), 
                                'Geral' : round(np.mean(notas_geral))},
                               ignore_index=True)
    notas_x = []
    notas_y = []
    notas_z = []
    notas_geral = []
    inicio = row['Início']
    fim = row['Fim']

In [0]:
df_quest.to_pickle("drive/My Drive/Udesc/questionarios.pkl")